# Store Embeddings for a Retrieval Augmented Generation (RAG) Use Case

RAG is especially useful for question-answering use cases that involve large amounts of unstructured documents containing important information. 

Let’s implement a RAG use case so that the next time you ask about the [orchestration service](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html), you get the correct response! To achieve this, you need to vectorize our context documents. You can find the documents to vectorize and store as embeddings in SAP HANA Cloud Vector Engine in the `documents` directory.

## LangChain

The Generative AI Hub Python SDK is compatible with the [LangChain](https://python.langchain.com/v0.1/docs/get_started/introduction) library. LangChain is a tool for building applications that utilize large language models, such as GPT models. It is valuable because it helps manage and connect different models, tools, and data, simplifying the process of creating complex AI workflows.

In [1]:
import json
import os
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)
 
# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "grounding"
    }
 
    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)
 
# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )
 
ai_core_client = create_ai_core_client(credCF)

https://israel-fsvdxbsq.authentication.eu11.hana.ondemand.com/oauth/token
sb-49ec08a9-d325-4480-9418-ad8801558203!b28574|aicore!b18
96d2ba69-3289-4190-ad82-c174e50f9f17$8C_adlgCYD6AscPgIKtLXJkIj1AL6i8p9Opw1JJZ0o8=
https://api.ai.prodeuonly.eu-central-1.aws.ml.hana.ondemand.com/v2
grounding


In [2]:
#!pip show generative-ai-hub-sdk

In [3]:
#!pip install "generative-ai-hub-sdk[all]"

In [4]:
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

In [5]:
import warnings
warnings.filterwarnings("ignore", message="As the c extension") #Avoid a warning message for "RuntimeWarning: As the c extension couldn't be imported, `google-crc32c` is using a pure python implementation that is significantly slower. If possible, please configure a c build environment and compile the extension. warnings.warn(_SLOW_CRC32C_WARNING, RuntimeWarning)"

# OpenAIEmbeddings to create text embeddings
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings

# TextLoader to load documents
from langchain_community.document_loaders import PyPDFDirectoryLoader

# different TextSplitters to chunk documents into smaller text chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangChain & HANA Vector Engine
from langchain_community.vectorstores.hanavector import HanaDB

👉 Change the `EMBEDDING_DEPLOYMENT_ID` in [variables.py](variables.py) to your deployment ID from exercise [01-explore-genai-hub](01-explore-genai-hub.md). For that go to **SAP AI Launchpad** application and navigate to **ML Operations** > **Deployments**.

☝️ The `EMBEDDING_DEPLOYMENT_ID` is the embedding model that creates vector representations of your text e.g. **text-embedding-3-small**.

👉 In [variables.py](variables.py) also set the `EMBEDDING_TABLE` to `"EMBEDDINGS_CODEJAM_>add your name here<"`, like `"EMBEDDINGS_CODEJAM_NORA"`.

👉 In the root folder of the project create a [.user.ini](../.user.ini) file with the HANA login information provided by the instructor.
```sh
[hana]
url=XXXXXX.hanacloud.ondemand.com
user=XXXXXX
passwd=XXXXXX
port=443
```

In [6]:
EMBEDDING_DEPLOYMENT_ID = "d1ae8a45e3d60115"

In [7]:
def get_hana_connection():
    conn = dbapi.connect(
        address='ec41b786-96de-467b-9ff5-db725945f89c.hna0.prod-us10.hanacloud.ondemand.com',
        port='443',
        user='DBADMIN',
        password='9hEW4UK86Fdt',
        encrypt=True
    )
    return conn

In [8]:
#!pip install hdbcli

In [9]:
from hdbcli import dbapi

# Chunking of the documents

Before you can create embeddings for your documents, you need to break them down into smaller text pieces, called "`chunks`". You will use the simplest chunking technique, which involves splitting the text based on character length and the separator `"\n\n"`, using the [Character Text Splitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/character_text_splitter/) from LangChain.

## Character Text Splitter

In [10]:
#!pip install pypdf

In [11]:

# Load custom documents
loader = PyPDFDirectoryLoader('documents/')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
print(f"Number of document chunks: {len(texts)}")


Number of document chunks: 59


Now you can connect to your SAP HANA Cloud Vector Engine and store the embeddings for your text chunks.

In [12]:
import importlib
#variables = importlib.reload(variables)

# DO NOT CHANGE THIS LINE BELOW - It is only to check whether you changed the table name in variables.py
#assert variables.EMBEDDING_TABLE != 'EMBEDDINGS_CODEJAM_ADD_YOUR_NAME_HERE', 'EMBEDDING_TABLE name not changed in variables.py'

# Create embeddings for custom documents
embeddings = OpenAIEmbeddings(deployment_id=EMBEDDING_DEPLOYMENT_ID)

db = HanaDB(
    embedding=embeddings, connection=get_hana_connection(), table_name="EMBEDDINGS_CODEJAM_"+"Testing"
)

# Delete already existing documents from the table
db.delete(filter={})

# add the loaded document chunks
db.add_documents(texts)
print(f"Table {db.table_name} created in the SAP HANA database.")

Table EMBEDDINGS_CODEJAM_Testing created in the SAP HANA database.


## Check the embeddings in SAP HANA Cloud Vector Engine

👉 Print the rows from your embedding table and scroll to the right to see the embeddings.

In [13]:
from IPython.display import Markdown
cursor = db.connection.cursor()

# Use `db.table_name` instead of `variables.EMBEDDING_TABLE` because HANA driver sanitizes a table name by removing unaccepted characters
is_ok = cursor.execute(f'''SELECT "VEC_TEXT", "VEC_META", TO_NVARCHAR("VEC_VECTOR") FROM "{db.table_name}"''')
record_columns=cursor.fetchone()
if record_columns:
    display({"VEC_TEXT" : record_columns[0], "VEC_META" : eval(record_columns[1]), "VEC_VECTOR" : record_columns[2]})
cursor.close()

{'VEC_TEXT': '/Examples/Orchestration Service\nOrchestration Service\nThis notebook demonstrates how to use the SDK to interact with the Orchestration\nService, enabling the creation of AI-driven workflows by seamlessly integrating various\nmodules, such as templating, large language models (LLMs), data masking and content\nfiltering. By leveraging these modules, you can build complex, automated workflows that\nenhance the capabilities of your AI solutions. For more details on configuring and using',
 'VEC_META': {'producer': 'Skia/PDF m131',
  'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
  'creationdate': '2025-02-02T21:04:28+00:00',
  'title': 'Orchestration Service | Generative AI hub SDK 4.1.1 documentation',
  'moddate': '2025-02-02T21:04:28+00:00',
  'source': 'documents\\Orchestration_Service_Generative_AI_hub_SDK.pdf',
  'total_pages': 14,
  'page': 0,
  'page_label': '1'},
 'VEC_VECTOR': '[-

[Next exercise](06-RAG.ipynb)